In [1]:
import pandas as pd
import numpy as np

In [2]:
google_id = '1teoxduT3bJAkrD0k1YIPn-mbVI83oQox'
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id=google_id,
                                    dest_path = './dados_cvm.csv',
                                    showsize = True)


google_id = '1J0fF-82tDg70wjaNg2Q8OGxPikA7phqp'
gdd.download_file_from_google_drive(file_id=google_id,
                                    dest_path = './dados_classes.csv',
                                    showsize = True)

82.9 MiB Done.
15.0 MiB Done.


In [3]:
df_1 = pd.read_csv('dados_cvm.csv')
df_2 = pd.read_csv('dados_classes.csv')
df = pd.merge(df_1, df_2, on='SK_Documento', how='inner')

<ipython-input-3-83f4cfa87e7c>:1: DtypeWarning: Columns (12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1 = pd.read_csv('dados_cvm.csv')


# Passo 1: Agregar cotas em Subordinadas e Senior por fundo

In [4]:
print(f"{len(df.loc[df['Patrimonio_Liquido'] == 0]) / len(df.index) * 100}% dos fundos tem PL == 0")

1.281484012254751% dos fundos tem PL == 0


In [5]:
print(f"{len(df.loc[df['Patrimonio_Liquido'] < 0]) / len(df.index) * 100}% dos fundos tem PL < 0")

0.09073444492363185% dos fundos tem PL < 0


In [6]:
print(f"Pré-operação tinhamos {len(df.columns)} colunas e {len(df.index)} linhas")
print(f"Pré-operação tinhamos {df['Classe_Serie'].nunique()} dados únicos na coluna Classe_Serie")

df.loc[df['Classe_Serie'].str.contains("sub", case=False), 'Classe_Serie'] = "Subordinada"
df.loc[df['Classe_Serie'].str.contains("nior", case=False), 'Classe_Serie'] = "Senior"

print(f"Agora temos {len(df.columns)} colunas e {len(df.index)} linhas")
print(f"Agora temos {df['Classe_Serie'].nunique()} dados únicos na coluna Classe_Serie")


Pré-operação tinhamos 336 colunas e 178543 linhas
Pré-operação tinhamos 57 dados únicos na coluna Classe_Serie
Agora temos 336 colunas e 178543 linhas
Agora temos 2 dados únicos na coluna Classe_Serie


In [7]:
# Agora vamos criar colunas agregando as 'cols_to_sum' por Subordinada e Senior
classe_serie_types = ["Subordinada", "Senior"]
cols_to_sum = ["Numero_Cotistas", "Quantidade_Cotas", "Valor_Cota", "Rentabilidade", "Valor_Total_Captado", "Quantidade_Cotas_Emitidas", "Valor_Total_Resgates", "Quantidade_Cotas_Resgatadas", "Valor_A_Pagar", "Quantidade_Cotas_A_Resgatar", "Valor_Amortizado_Cota", "Valor_Total_Amortizacao", "Desempenho_Esperado", "Desempenho_Realizado"]
for classe_serie_type in classe_serie_types:
  for col in cols_to_sum:
    df[col + "_" + classe_serie_type] = df[df['Classe_Serie'] == classe_serie_type].groupby('ID_Documento')[col].transform('sum')

print(f"Agora temos {len(df.columns)} colunas e {len(df.index)} linhas")

Agora temos 364 colunas e 178543 linhas


In [8]:
# Agora vamos remover fileiras duplicadas
df.drop(cols_to_sum, axis=1, inplace=True)
df = df.drop_duplicates()
print(f"Agora temos {len(df.columns)} colunas e {len(df.index)} linhas")

Agora temos 350 colunas e 111476 linhas


In [9]:
df = df.groupby("ID_Documento", as_index=False).agg("first")
print(f"Agora temos {len(df.columns)} colunas e {len(df.index)} linhas")

Agora temos 350 colunas e 57275 linhas


## Passo 2: Agregar Colunas por Prazo
Precisamos reduzir o número de colunas, o primeiro passo para isso é juntar as colunas que envolvem prazo da seguinte maneira:

- Somar campos entre 0 e 90 dias como `Curto Prazo (CP)`
- Somar campos entre 90 e 365 dias como `Médio Prazo (MP)`
- Somar campos acima de 365 dias como `Longo Prazo (LP)`

In [10]:
# Create Aggregate Columns
step_2_pretreat_col_count = len(df.columns)
df['Carteira_Direitos_Aquisicao_Prazo_CP'] = df['Carteira_Direitos_Aquisicao_Prazo_1_30_dias'] + df['Carteira_Direitos_Aquisicao_Prazo_31_60_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_61_90_Dias']
df['Carteira_Direitos_Aquisicao_Prazo_MP'] = df['Carteira_Direitos_Aquisicao_Prazo_91_120_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_121_150_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_151_180_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_181_360_Dias']
df['Carteira_Direitos_Aquisicao_Prazo_LP'] = df['Carteira_Direitos_Aquisicao_Prazo_361_720_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_721_1080_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_Acima_1080_Dias']
df['Carteira_Direitos_Aquisicao_Inadimplentes_CP'] = df['Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_31_60_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_61_90_Dias']
df['Carteira_Direitos_Aquisicao_Inadimplentes_MP'] = df['Carteira_Direitos_Aquisicao_Inadimplentes_91_120_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_121_150_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_151_180_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_181_360_Dias']
df['Carteira_Direitos_Aquisicao_Inadimplentes_LP'] = df['Carteira_Direitos_Aquisicao_Inadimplentes_361_720_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_721_1080_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_Acima_1080_Dias']
df['Carteira_Direitos_Aquisicao_Pagos_CP'] = df['Carteira_Direitos_Aquisicao_Pagos_1_30_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_31_60_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_61_90_Dias']
df['Carteira_Direitos_Aquisicao_Pagos_MP'] = df['Carteira_Direitos_Aquisicao_Pagos_91_120_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_121_150_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_151_180_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_181_360_Dias']
df['Carteira_Direitos_Aquisicao_Pagos_LP'] = df['Carteira_Direitos_Aquisicao_Pagos_361_720_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_721_1080_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_Acima_1080_Dias']
df['Carteira_Direitos_Sem_Aquisicao_Prazo_CP'] = df['Carteira_Direitos_Sem_Aquisicao_Prazo_1_30_dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_31_60_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_61_90_Dias']
df['Carteira_Direitos_Sem_Aquisicao_Prazo_MP'] = df['Carteira_Direitos_Sem_Aquisicao_Prazo_91_120_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_121_150_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_151_180_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_181_360_Dias']
df['Carteira_Direitos_Sem_Aquisicao_Prazo_LP'] = df['Carteira_Direitos_Sem_Aquisicao_Prazo_361_720_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_721_1080_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_Acima_1080_Dias']
df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_CP'] = df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_31_60_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_61_90_Dias']
df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_MP'] = df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_91_120_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_121_150_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_151_180_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_181_360_Dias']
df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_LP'] = df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_361_720_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_721_1080_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_Acima_1080_Dias']
df['Carteira_Direitos_Sem_Aquisicao_Pagos_CP'] = df['Carteira_Direitos_Sem_Aquisicao_Pagos_1_30_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_31_60_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_61_90_Dias']
df['Carteira_Direitos_Sem_Aquisicao_Pagos_MP'] = df['Carteira_Direitos_Sem_Aquisicao_Pagos_91_120_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_121_150_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_151_180_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_181_360_Dias']
df['Carteira_Direitos_Sem_Aquisicao_Pagos_LP'] = df['Carteira_Direitos_Sem_Aquisicao_Pagos_361_720_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_721_1080_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_Acima_1080_Dias']
df['Liquidez_CP'] = df['Liquidez_Ate_30_Dias'] + df['Liquidez_Ate_60_Dias'] + df['Liquidez_Ate_90_Dias']
df['Liquidez_MP'] = df['Liquidez_Ate_180_Dias'] + df['Liquidez_Ate_360_Dias']
df['Liquidez_LP'] = df['Liquidez_Acima_360_Dias']

agg_cols_to_drop = ['Carteira_Direitos_Aquisicao_Prazo_1_30_dias', 'Carteira_Direitos_Aquisicao_Prazo_31_60_Dias', 'Carteira_Direitos_Aquisicao_Prazo_61_90_Dias', 'Carteira_Direitos_Aquisicao_Prazo_91_120_Dias', 'Carteira_Direitos_Aquisicao_Prazo_121_150_Dias', 'Carteira_Direitos_Aquisicao_Prazo_151_180_Dias', 'Carteira_Direitos_Aquisicao_Prazo_181_360_Dias', 'Carteira_Direitos_Aquisicao_Prazo_361_720_Dias', 'Carteira_Direitos_Aquisicao_Prazo_721_1080_Dias', 'Carteira_Direitos_Aquisicao_Prazo_Acima_1080_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_31_60_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_61_90_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_91_120_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_121_150_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_151_180_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_181_360_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_361_720_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_721_1080_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_Acima_1080_Dias', 'Carteira_Direitos_Aquisicao_Pagos_1_30_Dias', 'Carteira_Direitos_Aquisicao_Pagos_31_60_Dias', 'Carteira_Direitos_Aquisicao_Pagos_61_90_Dias', 'Carteira_Direitos_Aquisicao_Pagos_91_120_Dias', 'Carteira_Direitos_Aquisicao_Pagos_121_150_Dias', 'Carteira_Direitos_Aquisicao_Pagos_151_180_Dias', 'Carteira_Direitos_Aquisicao_Pagos_181_360_Dias', 'Carteira_Direitos_Aquisicao_Pagos_361_720_Dias', 'Carteira_Direitos_Aquisicao_Pagos_721_1080_Dias', 'Carteira_Direitos_Aquisicao_Pagos_Acima_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_1_30_dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_31_60_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_61_90_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_91_120_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_121_150_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_151_180_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_181_360_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_361_720_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_721_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_Acima_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_31_60_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_61_90_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_91_120_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_121_150_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_151_180_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_181_360_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_361_720_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_721_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_Acima_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_1_30_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_31_60_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_61_90_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_91_120_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_121_150_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_151_180_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_181_360_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_361_720_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_721_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_Acima_1080_Dias', 'Liquidez_Ate_30_Dias', 'Liquidez_Ate_60_Dias', 'Liquidez_Ate_90_Dias', 'Liquidez_Ate_180_Dias', 'Liquidez_Ate_360_Dias', 'Liquidez_Acima_360_Dias']
df.drop(agg_cols_to_drop, axis=1, inplace=True)
step_2_posttreat_col_count = len(df.columns)

print(f"Dropamos {step_2_pretreat_col_count - step_2_posttreat_col_count} colunas")
print(f"Agora temos {len(df.columns)} colunas e {len(df.index)} linhas")

Dropamos 45 colunas
Agora temos 305 colunas e 57275 linhas


# Passo 3: Dropas colunas onde todos os valores são iguais

In [11]:
# Dropar Colunas onde todos os valores são iguais
nunique = df.nunique()
cols_to_drop = nunique[nunique == 1].index

print(f"Dropando {len(cols_to_drop)} colunas onde todos os valores são iguais: ")
print(cols_to_drop)
df = df.drop(cols_to_drop, axis=1)

print(f"Agora temos {len(df.columns)} colunas e {len(df.index)} linhas")

Dropando 15 colunas onde todos os valores são iguais: 
Index(['Ativo_Direitos_Sem_Aquisicao_Creditos_Performar',
       'Ativo_Direitos_Sem_Aquisicao_Creditos_Empresas_Recuperacao',
       'Ativo_Direitos_Sem_Aquisicao_Creditos_Receitas_Publicas',
       'Ativo_Direitos_Sem_Aquisicao_Creditos_Fator_Risco',
       'Ativo_Coberturas_Prestadas', 'Carteira_Creditos_Tributarios',
       'Carteira_Royalties', 'Taxas_Titulos_Federais_Juros_Venda_Minina',
       'Taxas_Titulos_Federais_Juros_Venda_Maxima',
       'Taxas_CDB_Desconto_Venda_Minina',
       'Taxas_CDB_Desconto_Venda_Media_Ponderada',
       'Taxas_CDB_Desconto_Venda_Maxima', 'Taxas_CDB_Juros_Venda_Minina',
       'Taxas_CDB_Juros_Venda_Media_Ponderada',
       'Taxas_CDB_Juros_Venda_Maxima'],
      dtype='object')
Agora temos 290 colunas e 57275 linhas


# Passo 4: Somar outros ativos em uma coluna resumida
Aqui vamos somar as seguintas colunas em `Ativo_Outros`

- `Ativo_Valores_Mobiliarios`
- `Ativo_Titulos_Federais`
- `Ativo_CDB`
- `Ativo_Operacoes_Compromissadas`
- `Ativo_Outros_Ativos_Renda_Fixa`
- `Ativo_Cotas_FIDC`
- `Ativo_Warrants`
- `Ativo_Posicao_Derivativos`
- `Ativo_Mercado_Termo`
- `Ativo_Mercado_Opcoes`
- `Ativo_Mercado_Futuro`
- `Ativo_Diferencial_Swap`
- `Ativo_Depositos_Margem`

In [12]:
# Vamos somar outros ativos
ativos_drop = ["Ativo_Valores_Mobiliarios", "Ativo_Titulos_Federais", "Ativo_CDB", "Ativo_Operacoes_Compromissadas", "Ativo_Outros_Ativos_Renda_Fixa", "Ativo_Cotas_FIDC", "Ativo_Warrants", "Ativo_Posicao_Derivativos", "Ativo_Mercado_Termo", "Ativo_Mercado_Opcoes", "Ativo_Mercado_Futuro", "Ativo_Diferencial_Swap", "Ativo_Depositos_Margem", "Ativo_Debentures", "Ativo_CRI", "Ativo_Notas_Promissorias", "Ativo_Letras Financeiras", "Ativo_Cotas_Fundos_ICVM_555", "Ativo_Outros_Valores_Mobiliarios", "Ativo_Cotas_FIDC_NP", "Ativo_Provisao_Debentures_CRI_NP_LF", "Ativo_Provisao_Cotas_FIDC", "Ativo_Provisao_Outros_Ativos"]

for ativo in ativos_drop:
  df["Ativo_Outros"] = df["Ativo_Outros"] +  df[ativo]

df = df.drop(ativos_drop, axis=1)

print(f"Dropamos {len(ativos_drop)} colunas")
print(f"Agora temos {len(df.columns)} colunas e {len(df.index)} linhas")

Dropamos 23 colunas
Agora temos 267 colunas e 57275 linhas


# Passo 5: Criar uma coluna de `Provisao`

In [13]:
df["Provisao"] = df["Ativo_Direitos_Aquisicao_Provisao_Reducao"] + df["Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao"]
print(f"Agora temos {len(df.columns)} colunas e {len(df.index)} linhas")

Agora temos 268 colunas e 57275 linhas


# Manter apenas as colunas selecionadas

In [14]:
columns_to_keep = ["CNPJ", "Data_Competencia", "Patrimonio_Liquido", "CNPJ_Administrador", "Forma_Condominio", "Carteira_Direitos_Aquisicao_Inadimplentes_CP", "Carteira_Direitos_Aquisicao_Inadimplentes_MP", "Carteira_Direitos_Aquisicao_Inadimplentes_LP", "Carteira_Direitos_Sem_Aquisicao_Inadimplentes_CP", "Carteira_Direitos_Sem_Aquisicao_Inadimplentes_MP", "Carteira_Direitos_Sem_Aquisicao_Inadimplentes_LP", "Liquidez_CP", "Liquidez_MP", "Liquidez_LP", "Carteira_Financeiro", "Provisao", "Quantidade_Cotas_Senior", "Valor_Cota_Senior", "Rentabilidade_Senior", "Valor_Total_Captado_Senior", "Valor_Total_Resgates_Senior", "Valor_A_Pagar_Senior", "Quantidade_Cotas_A_Resgatar_Senior", "Valor_Amortizado_Cota_Senior", "Valor_Total_Amortizacao_Senior", "Desempenho_Esperado_Senior", "Desempenho_Realizado_Senior", "Quantidade_Cotas_Subordinada", "Valor_Cota_Subordinada", "Rentabilidade_Subordinada", "Valor_Total_Captado_Subordinada", "Valor_Total_Resgates_Subordinada", "Valor_A_Pagar_Subordinada", "Quantidade_Cotas_A_Resgatar_Subordinada", "Valor_Amortizado_Cota_Subordinada", "Valor_Total_Amortizacao_Subordinada", "Desempenho_Esperado_Subordinada", "Desempenho_Realizado_Subordinada"]
df = df[columns_to_keep]

print(f"Agora temos {len(df.columns)} colunas e {len(df.index)} linhas")

Agora temos 38 colunas e 57275 linhas


# Calcular índice de subordinação

Índice de Subordinação = Valor Total das Cotas Subordinadas / PL do Fundo